## MISA (2024-2025)
- Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

- Izay misy hoe `YOUR CODE HERE` na `YOUR ANSWER HERE` ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
chatgpt youtube

---

# Ridge regression

In [1]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.linear_model import Ridge 

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = datasets.load_diabetes()
X_train, y_train = data.data, data.target

### Inverse

In [2]:
def fit_inverse(X, y, alpha=1.0, fit_intercept=False):
    """Méthode directe utilisant l'inverse pour la régression de Ridge"""
    
    # Ajouter l'intercept si nécessaire
    if fit_intercept:
        X = np.c_[np.ones(X.shape[0]), X]  # Ajouter une colonne de 1 pour l'intercept
    
    # Calculer la matrice X^T X + alpha * I
    I = np.eye(X.shape[1])  # Matrice identité de taille (nombre de caractéristiques + 1 si intercept)
    if fit_intercept:
        I[0, 0] = 0  # Ne pas régulariser l'intercept
    
    # Résolution de l'équation de régression de Ridge
    beta = np.dot(np.linalg.inv(X.T @ X + alpha * I) , np.dot(X.T, y))
    
    return beta

In [3]:
w = fit_inverse(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-11

prediction error:  2.909795356615573e-14


### QR using QR linear regression

In [4]:
def fit_qr(X, y):
    """ QR approach """
    # Décomposition QR de X
    Q, R = np.linalg.qr(X)
    
    # Résoudre pour beta en utilisant R et Q^T y
    Qt_y = np.dot(Q.T, y)  # Calculer Q^T y
    beta = np.linalg.solve(R, Qt_y)  # Résoudre R beta = Q^T y
    
    return beta


def fit_qr(X, y, alpha=1.0, fit_intercept=False):
    """
    Régression Ridge utilisant la décomposition QR.
    
    Arguments:
    - X : Matrice des caractéristiques (numpy array)
    - y : Vecteur cible (numpy array)
    - alpha : Paramètre de régularisation (float)
    - fit_intercept : Booléen, indique si l'intercept doit être ajusté.
    
    Retourne:
    - beta : Coefficients de la régression Ridge.
    """
    # Ajouter une colonne pour l'intercept si nécessaire
    if fit_intercept:
        X = np.c_[np.ones(X.shape[0]), X]
    
    # Décomposition QR
    Q, R = np.linalg.qr(X)
    
    # Construction de la matrice régularisée
    I = np.eye(R.shape[1])  # Matrice identité
    if fit_intercept:
        I[0, 0] = 0  # Ne pas régulariser l'intercept

    # Résolution de (R.T @ R + alpha * I) beta = R.T @ Q.T @ y
    beta = np.linalg.solve(np.dot(R.T , R) + alpha * I, R.T @ Q.T @ y)
    
    return beta


In [5]:
w = fit_qr(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-11

prediction error:  5.895941028710679e-14


### SVD

In [11]:
def fit_svd(X, y, alpha=1.0, fit_intercept=False):
    """SVD approach"""
    if fit_intercept:
        # Ajouter une colonne de 1 à X pour le terme d'interception
        X = np.c_[np.ones(X.shape[0]), X]

    # Calculer X^T X et X^T y
    A = np.dot(X.T, X)  # Matrice de covariances X^T X
    b = np.dot(X.T, y)  # Matrice des produits X^T y

    # Appliquer la régularisation (alpha * I)
    I = np.eye(X.shape[1])  # Matrice identité de la même dimension que X^T X
    if fit_intercept:
        I[0, 0] = 0 
    A_reg = A + alpha * I  # Matrice régulière (avec alpha pour la régularisation Ridge)

    # Appliquer la décomposition SVD sur A_reg
    U, S, Vt = np.linalg.svd(A_reg)

    # Calcul de l'inverse de S
    S_inv = np.diag(1.0 / S)

    # Calculer la solution (Vt.T @ S_inv @ U.T @ b)
    theta = np.dot(Vt.T, np.dot(S_inv, np.dot(U.T, b)))

    return theta


In [12]:
w = fit_svd(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-11

prediction error:  1.7484222244850787e-13


## Everything in a class

In [13]:
class RidgeRegression():
    def __init__(self, fit_intercept=True, method="inverse", alpha=1.0):
        self.w = None
        self.fit_intercept = fit_intercept
        self.method = method
        self.alpha = alpha
    
    def fit(self, X, y):
        # Choisir la méthode en fonction de l'attribut `method`
        if self.method == "inverse":
            self.w = fit_inverse(X, y, alpha=self.alpha, fit_intercept=self.fit_intercept)
        elif self.method == "qr":
            self.w = fit_qr(X, y, alpha=self.alpha, fit_intercept=self.fit_intercept)
        elif self.method == "svd":
            self.w = fit_svd(X, y, alpha=self.alpha, fit_intercept=self.fit_intercept)
        else:
            raise ValueError("Méthode non reconnue. Choisissez parmi 'inverse', 'qr', 'svd'.")
    
    def predict(self, X):
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]  # Ajouter une colonne de 1 pour l'interception
        return np.dot(X, self.w)  # Calcul des prédictions


## without bias

In [14]:
# OTHER APPROACHES
sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = RidgeRegression(fit_intercept=False, method="inverse", alpha=0.1)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
print("prediction error inverse: ", error)
assert error <= 1e-11

model_qr = RidgeRegression(fit_intercept=False, method="qr", alpha=0.1)
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
print("prediction error qr: ", error_qr)
assert error_qr <= 1e-11

model_svd = RidgeRegression(fit_intercept=False, method="svd", alpha=0.1)
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
print("prediction error svd: ", error_svd)
assert error_svd <= 1e-11

prediction error inverse:  2.1335231184290413e-14
prediction error qr:  5.067117406268825e-14
prediction error svd:  1.90164917840323e-13


## with bias

In [15]:
# OTHER APPROACHES
sk_model = Ridge(fit_intercept=True, alpha=0.1)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = RidgeRegression(fit_intercept=True, method="inverse", alpha=0.1)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
print("prediction error inverse: ", error)
assert error <= 1e-11

model_qr = RidgeRegression(fit_intercept=True, method="qr", alpha=0.1)
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
print("prediction error qr: ", error_qr)
assert error_qr <= 1e-11

model_svd = RidgeRegression(fit_intercept=True, method="svd", alpha=0.1)
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
print("prediction error svd: ", error_svd)
assert error_svd <= 1e-11

prediction error inverse:  1.8372972587190376e-15
prediction error qr:  1.3779729440392789e-15
prediction error svd:  2.544290727651881e-15
